In [378]:
import importlib
import func
importlib.reload(func)
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

## BWSC101

In [379]:
df_101191 = pd.read_excel("data/191_BWSC101 Release Log Form.xlsx")
df_101592 = pd.read_excel("data/592_BWSC101 Release Log Form.xlsx")
df_101607 = pd.read_excel("data/607_BWSC101 Release Log Form.xlsx")

In [380]:
print(df_101191.shape)
print(df_101592.shape)
print(df_101607.shape)

(10943, 215)
(1535, 223)
(7125, 250)


In [381]:
df_101191["RTN"] = df_101191.apply(func.completeRTN, axis=1)
df_101592["RTN"] = df_101592.apply(func.completeRTN, axis=1)
df_101607["RTN"] = df_101607.apply(func.completeRTN, axis=1)

In [382]:
func.preprocess(df_101607, "101607proc.xlsx", "101607")
print(df_101607.shape)
func.preprocess(df_101592, "101592proc.xlsx", "101592")
print(df_101592.shape)
func.preprocess(df_101191, "101191proc.xlsx", "101191")
print(df_101191.shape)

(7125, 144)
(1535, 144)
(10943, 144)


In [383]:
df_101 = df_101191.append(df_101592)
df_101 = df_101.append(df_101607)

In [384]:
df_101.shape

(19603, 144)

In [385]:
# As asked, we should only keep the data of which A2A or A3A is checked.
df_101 = df_101[(df_101["A3A"] == 1) | (df_101["A2A"] == 1)]

In [386]:
df_101.shape

(13982, 144)

## TClass

In [387]:
tclass = pd.read_excel('data/TClass Phase Action Dates All RTNs mgf A 04-10-2018.xlsm', sheetname="All")

In [388]:
df_tclass = tclass

In [389]:
df_tclass = df_tclass.set_index("RTN")

In [390]:
df_tclass["Status"].unique()

array(['ADQREG', 'LSPNFA', 'DEPNFA', 'DEPNDS', 'RAO', 'PENNFA', 'PENNDS',
       'URAM', 'RAONR', 'TIERII', 'WCSPRM', 'TIERI', 'TIER1D', 'REMOPS',
       'TCLASS', 'TMPS', 'ROSTRM', 'PSC', 'PSNC', 'DPS', 'DEPMOU',
       'STMRET', 'INVSUB', 'DPSTRM', 'UNCLSS', 'SPECPR'], dtype=object)

In [391]:
exclude_status = ['ADQREG', 'DEPMOU', 'DEPNDS', 'DEPNFA', 'DPS', 'DPSTRM', 'INVSUB', 'STMRET', 'URAM', 'UNCLSS']

In [392]:
df_tclass = df_tclass[(df_tclass["Notification"] >= "2006-06-01") & (df_tclass["Notification"] <= "2016-12-28")]

In [393]:
df_tclass.shape

(14792, 30)

In [394]:
df_tclass = df_tclass[~df_tclass["Status"].isin(exclude_status)]

In [395]:
df_tclass["Status"].unique()

array(['RAO', 'TIERII', 'PSNC', 'RAONR', 'TIER1D', 'TMPS', 'REMOPS',
       'TIERI', 'PSC', 'ROSTRM', 'SPECPR', 'TCLASS'], dtype=object)

In [396]:
df_tclass.shape

(13843, 30)

In [397]:
drop_index = df_tclass[(df_tclass["Status"].isin(["REMOPS", "ROSTRM", "TCLASS", "TIERI", "TIERII"])) & df_tclass["Phase1Cs"].isnull() & df_tclass["RaoNr"].isnull()].index

In [398]:
df_tclass = df_tclass.drop(drop_index, axis=0)

In [399]:
df_tclass["length"] = df_tclass.apply(func.daylength, axis=1)

In [400]:
# df_tclass[df_tclass["length"] < 0].to_excel("less than zero.xlsx")

In [401]:
df_tclass = df_tclass[df_tclass["length"] >= 0]

In [402]:
df_tclass.shape

(13796, 31)

In [403]:
df_tclass["Tier1D"] = df_tclass.apply(func.isTier1D, axis=1)

In [404]:
df_tclass["Tier1D"].groupby(df_tclass["Tier1D"]).count()

Tier1D
Non-Tier1D    11152
Tier1D         2644
Name: Tier1D, dtype: int64

In [405]:
df_tier = df_101.set_index("RTN").join(df_tclass.loc[:, ["Tier1D"]], how='inner')

In [406]:
# df_tier.to_excel("data/df_tier.xlsx")

In [407]:
df_tier.shape

(10888, 144)

### chemicals

In [350]:
df_chemicals = pd.read_excel("data/Chemical_Class_Features1.xlsx")

In [353]:
df_chemicals = df_chemicals.set_index("RTN")

In [356]:
df_chemicals.head()

,CAUSTIC,CVOC,ERROR,METAL,O/HM,OTHER,PAH,PCB,PETROLEUM,UNKNOWN,VOC
RTN,,,,,,,,,,,
2-0010055,1,0,0,0,0,0,0,0,0,0,0
3-0010061,0,0,0,0,0,0,0,0,1,0,0
3-0010034,0,0,0,0,0,0,0,0,1,0,0
4-3010035,0,0,0,0,0,0,0,0,1,0,0
3-0010037,0,0,0,0,0,0,0,0,1,0,0


In [362]:
list_chemicals = df_chemicals.columns.tolist()

In [364]:
dict_chemicals = {}
for che in list_chemicals:
    dict_chemicals[che] = 0.0

In [369]:
df_tierc = df_tier.join(df_chemicals, how='left')
df_tierc = df_tierc.fillna(dict_chemicals)

In [372]:
df_tierc.to_excel("data/df_tierc.xlsx")

## IRA

In [304]:
df_ira = pd.read_csv("data/RTN_IRA.csv")
df_ira.drop("Unnamed: 0", axis=1, inplace=True)

In [306]:
df_ira = df_101.set_index("RTN").join(df_ira.set_index("rtn"), how='inner')

In [311]:
df_ira["IRA"].groupby(df_ira["IRA"]).count()

IRA
IRA_Not_Present    6929
IRA_Present        3967
Name: IRA, dtype: int64

In [335]:
df_ira.to_excel("data/df_ira.xlsx")

## Including GIS

In [336]:
CENSUS_2010_B = pd.read_excel("data/Intersect_Release_CENSUS_2010_Blocks.xls")
T_group_quarters_pop                = pd.read_excel("data/GIS/TRACT/group_quarters_pop.xlsx")
T_households_by_age_family_children = pd.read_excel("data/GIS/TRACT/households_by_age_family_children.xlsx")
T_households_size_by_family = pd.read_excel("data/GIS/TRACT/households_size_by_family.xlsx")
T_housing_owner_rental_demographics = pd.read_excel("data/GIS/TRACT/housing_owner_rental_demographics.xlsx")
T_housing_residency_characteristics = pd.read_excel("data/GIS/TRACT/housing_residency_characteristics.xlsx")
T_pop_by_age_gender = pd.read_excel("data/GIS/TRACT/pop_by_age_gender.xlsx")
T_pop_by_race = pd.read_excel("data/GIS/TRACT/pop_by_race.xlsx")

In [337]:
CENSUS_2010_B = CENSUS_2010_B.loc[:, ["rtn", "GEOID10"]]
gis_block = T_group_quarters_pop.join(T_households_by_age_family_children.set_index("LOGRECNO"), how='inner', on='LOGRECNO', rsuffix='_')
gis_block = gis_block.join(T_households_size_by_family.set_index("LOGRECNO"), how='inner', on='LOGRECNO', rsuffix='_')
gis_block = gis_block.join(T_housing_owner_rental_demographics.set_index("LOGRECNO"), how='inner', on='LOGRECNO', rsuffix='_')
gis_block = gis_block.join(T_housing_residency_characteristics.set_index("LOGRECNO"), how='inner', on='LOGRECNO', rsuffix='_')
gis_block = gis_block.join(T_pop_by_age_gender.set_index("LOGRECNO"), how='inner', on='LOGRECNO', rsuffix='_')
gis_block = gis_block.join(T_pop_by_race.set_index("LOGRECNO"), how='inner', on='LOGRECNO', rsuffix='_')

In [338]:
CENSUS_2010_B['GEOID10'] = CENSUS_2010_B['GEOID10'].apply(lambda x : str(x)[:11])
gis_block["GEOID10"] = gis_block["GEOID10"].astype(str)
gis_block = CENSUS_2010_B.join(gis_block.set_index("GEOID10"), how='inner', on="GEOID10", rsuffix='_')
gis_block = gis_block.set_index("rtn")
features = gis_block.columns.tolist()
features = list(filter(lambda a: a != 'GEOID10_', features))
features = list(filter(lambda a: a != 'GEOID10', features))
features = list(filter(lambda a: a != 'LOGRECNO', features))
gis_block = gis_block.loc[:, features]

In [340]:
gis_block.head()

,GQ_POP,GQ_I_POP,GQ_I_CRRCT,GQ_I_JUVNL,GQ_I_NRSNG,GQ_I_OTHR,GQ_NI_POP,GQ_NI_UNIV,GQ_NI_MIL,GQ_NI_OTHR,...,NH_OTHER,NH_MULTI,HISP,H_WHITE,H_BLACK,H_NATV,H_ASN,H_ISLND,H_OTHER,H_MULTI
rtn,,,,,,,,,,,,,,,,,,,,,
4-0022830,4,0,0,0,0,0,4,0,0,4,...,13,36,302,253,6,0,0,0,35,8
4-0025038,4,0,0,0,0,0,4,0,0,4,...,13,36,302,253,6,0,0,0,35,8
4-0021874,4,0,0,0,0,0,4,0,0,4,...,13,36,302,253,6,0,0,0,35,8
4-0025255,4,0,0,0,0,0,4,0,0,4,...,13,36,302,253,6,0,0,0,35,8
4-0022428,6,0,0,0,0,0,6,0,0,6,...,55,98,602,432,17,0,0,0,136,17


In [341]:
df_tier_gis = df_tier.join(gis_block, how='inner')

In [343]:
df_tier.shape

(10888, 144)

In [345]:
df_tier_gis.to_excel("df_tier_gis.xlsx")

In [374]:
df_tier_gis.shape

(9591, 386)

In [375]:
df_tier_gisc = df_tier_gis.join(df_chemicals, how='left')
df_tier_gisc = df_tier_gisc.fillna(dict_chemicals)

In [376]:
df_tier_gisc.shape

(9591, 397)

In [377]:
df_tier_gisc.to_excel("data/df_tier_gisc.xlsx")

In [346]:
df_ira_gis = df_ira.join(gis_block, how='inner')

In [347]:
df_ira_gis.shape

(10898, 386)

In [348]:
df_ira_gis.to_excel("df_ira_gis.xlsx")